In [1]:
import psycopg2


Addresses of local dbs

In [2]:


# Database connection details
connection_params = {
    "dbname": "Vistara",
    "user": "postgres",
    "password": "addy",
    "host": "localhost",  # Use the actual host (e.g., "127.0.0.1" or server IP)
    "port": 5432          # Default PostgreSQL port
}

try:
    # Establish connection
    conn = psycopg2.connect(**connection_params)
    print("Connection successful")

    # Create a cursor object
    cursor = conn.cursor()

    # Execute a query
    # cursor.execute("""SELECT table_name, column_name, data_type 
    #                     FROM information_schema.columns
    #                     WHERE table_schema = 'public'
    #                     ORDER BY table_name, ordinal_position;""")
    cursor.execute("""SELECT 
    tc.table_schema AS referencing_schema,
    tc.table_name AS referencing_table,
    kcu.column_name AS referencing_column,
    ccu.table_schema AS referenced_schema,
    ccu.table_name AS referenced_table,
    ccu.column_name AS referenced_column
FROM 
    information_schema.table_constraints AS tc
JOIN 
    information_schema.key_column_usage AS kcu
    ON tc.constraint_name = kcu.constraint_name
    AND tc.table_schema = kcu.table_schema
JOIN 
    information_schema.constraint_column_usage AS ccu
    ON ccu.constraint_name = tc.constraint_name
    AND ccu.table_schema = tc.table_schema
WHERE 
    tc.constraint_type = 'FOREIGN KEY'
ORDER BY 
    referencing_schema, referencing_table;

 """)

    # Fetch and print the result
    version = cursor.fetchall()
    print(f"PostgreSQL version: {version}")

    # Close cursor and connection
    cursor.close()
    conn.close()
except Exception as e:
    print(f"Error: {e}")


Connection successful
PostgreSQL version: [('public', 'route', 'flightid', 'public', 'flightdetails', 'flightid')]


In [3]:
def jaro_winkler(string1, string2):
    """
    Custom implementation of the Jaro-Winkler similarity.
    :param string1: First string.
    :param string2: Second string.
    :return: Jaro-Winkler similarity score as a float.
    """
    def get_matching_characters(s1, s2, max_distance):
        """Find characters that match within max_distance."""
        matched = []
        matched_indices = set()
        for i, char1 in enumerate(s1):
            for j in range(max(0, i - max_distance), min(len(s2), i + max_distance + 1)):
                if char1 == s2[j] and j not in matched_indices:
                    matched.append(char1)
                    matched_indices.add(j)
                    break
        return matched

    def get_transpositions(s1_matches, s2_matches):
        """Count the number of transpositions between matched characters."""
        return sum(1 for c1, c2 in zip(s1_matches, s2_matches) if c1 != c2) // 2

    # Step 1: Jaro similarity
    max_distance = (max(len(string1), len(string2)) // 2) - 1
    string1_matches = get_matching_characters(string1, string2, max_distance)
    string2_matches = get_matching_characters(string2, string1, max_distance)

    if not string1_matches or not string2_matches:
        return 0.0

    m = len(string1_matches)
    t = get_transpositions(string1_matches, string2_matches)
    jaro = (m / len(string1) + m / len(string2) + (m - t) / m) / 3

    # Step 2: Jaro-Winkler adjustment
    prefix = 0
    for c1, c2 in zip(string1, string2):
        if c1 == c2:
            prefix += 1
        else:
            break
        if prefix == 4:  # Only consider up to 4 characters for prefix scaling
            break

    scaling_factor = 0.1  # Winkler's scaling factor
    jaro_winkler = jaro + prefix * scaling_factor * (1 - jaro)
    return jaro_winkler

# Example usage
score = jaro_winkler("martha", "marhta")
print(f"Jaro-Winkler Score: {score}")


Jaro-Winkler Score: 0.9611111111111111


In [4]:
def get_schema(connection_params):
    try:
        # Establish connection
        conn = psycopg2.connect(**connection_params)
        print("Connection successful")

        # Create a cursor object
        cursor = conn.cursor()

        # Execute a query
        cursor.execute("""SELECT table_name, column_name, data_type 
                            FROM information_schema.columns
                            WHERE table_schema = 'public'
                            ORDER BY table_name, ordinal_position;""")

        # Fetch and print the result
        results= cursor.fetchall()
        # print(f"PostgreSQL version: {version}")

        # Close cursor and connection
        cursor.close()
        conn.close()
        return results
    except Exception as e:
        print(f"Error: {e}")
        return None

    

In [5]:
fields={}

In [6]:
def custom_query(connection_params,query):
    try:
        # Establish connection
        conn = psycopg2.connect(**connection_params)
        print("Connection successful")

        # Create a cursor object
        cursor = conn.cursor()

        # Execute a query
        cursor.execute(f"{query}")

        # Fetch and print the result
        results= cursor.fetchall()
        # print(f"PostgreSQL version: {version}")
        column_names = [desc[0] for desc in cursor.description]
        fields[connection_params["dbname"]]=column_names

        # Close cursor and connection
        cursor.close()
        conn.close()
        return results
    except Exception as e:
        print(f"Error: {e}")
        return None

    

In [7]:
Connection_params={
    "Air India":{
    "dbname": "Air India",
    "user": "postgres",
    "password": "addy",
    "host": "localhost",  # Use the actual host (e.g., "127.0.0.1" or server IP)
    "port": 5432          # Default PostgreSQL port
    },
    "Vistara":{
    "dbname": "Vistara",
    "user": "postgres",
    "password": "addy",
    "host": "localhost",  # Use the actual host (e.g., "127.0.0.1" or server IP)
    "port": 5432          # Default PostgreSQL port
    },
    "Indigo":{
    "dbname": "Indigo",
    "user": "postgres",
    "password": "addy",
    "host": "localhost",  # Use the actual host (e.g., "127.0.0.1" or server IP)
    "port": 5432          # Default PostgreSQL port
    }
}

In [8]:
# query is select * from (all combined customer tables tables)

fkey_mapping={
    "Air India":"customerid",
    "Indigo":"emailid",
    "Vistara":""
}

In [9]:
combined_table_mapping={}

In [10]:
#dictionary for mapping 
table_mapping={
    "Air India":["customer_age","customer_info","customer_health"],
    "Indigo":["clients","loyalty"],
    "Vistara":["passenger"]
    
}

In [11]:
for k,v in table_mapping.items():
    joined_table=v[0]
    for i in range(1,len(v)):
        
        joined_table+=f" join {v[i]} on {v[i-1]}.{fkey_mapping[k]} = {v[i]}.{fkey_mapping[k]}"
    combined_table_mapping[k]=joined_table



In [ ]:
combined_table_mapping

In [29]:
#dictionary for all customer data:
cust_data={}
for k,v in Connection_params.items():
    query=f"select * from {combined_table_mapping[k]} ;"
    cust_data[k]=custom_query(v,query)
    
    

Connection successful
Connection successful
Connection successful


In [30]:
cust_data["Indigo"][0]

('Priya Singh',
 'priya.singh@example.com',
 '9876543215',
 datetime.date(1988, 11, 3),
 None,
 'priya.singh@example.com',
 1000)

In [ ]:
fields

global schema for customer-
Customer = [
    "Name", "Email", "Phone no.", "Point/frequent_flyer_miles", 
    "Age Group", "Medical Condition (optional)", 
    "City", "Address"
]

In [26]:
# mapping from schema matching
schema_mapping={
    "Air India":{
    
    "Name":"fullname", "Email":"emailaddress", "Phone no.":"contactnumber", "Point/frequent_flyer_miles":"loyaltypoints", 
    "Age":"agegroup", "City":"cityofresidence"
        
        
    },
    "Indigo":{
        "Name":"full_name", "Email":"emailid", "Phone no.":"contact", "Point/frequent_flyer_miles":"points", 
    "Age": "dob", 
    "City":"city"
    },
    "Vistara":{
          "Name":"passenger_name", "Email":"email", "Phone no.":"contactno", "Point/frequent_flyer_miles":"frequentflyerpoints", 
    "Age": "dateofbirth", 
    "City":"city"
    }


}

Blocking based on city

In [15]:
field_index={"Air India":{"City":-1},"Vistara":{"City":-1}}
field_index["Air India"]["City"]

-1

In [27]:
dbnames=["Air India","Vistara"]
#find fields
field_index={"Air India":{},"Vistara":{}}
# field_index={}

for db in dbnames:
    print(db)
    for i in range(len(fields[db])):
        if fields[db][i]==schema_mapping[db]["City"]:
            # print(field_index[db])
            print("going")
            field_index[db]["City"]=i
        elif fields[db][i]==schema_mapping[db]["Age"]:
            field_index[db]["Age"]=i
    

Air India
going
Vistara
going


In [23]:
field_index["Air India"]["City"]

KeyError: 'City'

In [31]:
cust_data.keys()

dict_keys(['Air India', 'Vistara', 'Indigo'])

In [33]:
# del cust_data['Indigo']
city_blocks={}
for db in dbnames:
    city_blocks[db]={}

for airline, data in cust_data.items():
    for row in data:
        if row[field_index[airline]["City"]] not in city_blocks[airline]:
            city_blocks[airline][row[field_index[airline]["City"]]]=[row]
        else:
            city_blocks[airline][row[field_index[airline]["City"]]].append(row)


blocks=city_blocks

for airline, cities in city_blocks.items():
    for city,rows in cities.items():
        blocks[airline][city]={}
        for row in rows:
            if row[field_index[airline]["Age"]] not in blocks[airline][city]:
                blocks[airline][city][row[field_index[airline]["Age"]]]=[row]
            else:
                blocks[airline][city][row[field_index[airline]["Age"]]].append(row)

            






In [35]:
blocks["Air India"]["Mumbai"]

{'26-35': [(1,
   '26-35',
   1,
   'John Doe',
   'john.doe@example.com',
   '9876543210',
   1200,
   'Mumbai',
   1,
   False),
  (25,
   '26-35',
   25,
   'Aditi Patel',
   'aditi.patel@example.com',
   '9876543236',
   1100,
   'Mumbai',
   25,
   False)],
 '18-25': [(11,
   '18-25',
   11,
   'Sara Ali',
   'sara.ali@example.com',
   '9876543222',
   600,
   'Mumbai',
   11,
   False),
  (19,
   '18-25',
   19,
   'Simran Kaur',
   'simran.kaur@example.com',
   '9876543230',
   700,
   'Mumbai',
   19,
   False),
  (31,
   '18-25',
   31,
   'Lata Mehta',
   'lata.mehta@example.com',
   '9876543242',
   1800,
   'Mumbai',
   31,
   True),
  (39,
   '18-25',
   39,
   'Farah Khan',
   'farah.khan@example.com',
   '9876543250',
   1700,
   'Mumbai',
   39,
   True),
  (47,
   '18-25',
   47,
   'Pranav Bhatia',
   'pranav.bhatia@example.com',
   '9876543258',
   1200,
   'Mumbai',
   47,
   False)]}